# [RL Autonomous Driver](https://github.com/tonyroumi/COGS188_RLAutonomousDriver)

## Group members

- Ivan Guo
- Ruiqi(Ricky) Zhu
- Anthony Roumi

# Abstract
This project implements a reinforcement learning approach to autonomous driving using the CARLA simulator. The goal was to develop an agent capable of driving in urban environments while adhering to traffic rules and avoiding collisions. The model uses an Advantage Actor-Crtic (A3C) architecture that processes RGB camera images as input states and outputs discrete vehicle control commands. The agent learns through reward functions that incetivize progress towards the destination while penalizing unsafe behaviors like speeding, collisions, traffic light violations, and lane-keeping. Training data is collected through interaction with the CARLA environment, where the agent explores using an epislon-greedy strategy. This solution utilizes convolutional neural networks for visual processing with prioritized experience replay to enhance sample efficiency. The primary learning mechanism relies on end-to-end training from pixels to actions. Evaluation metrics track collision rates, success rates, and cumulative rewards across training episodes. Given the challenging nature of the task, the agent struggles to adapt to driving scenarios without adequate training.

# Background

In recent years, autonomous driving has made great strides in the field of Artificial Intelligence (AI), particularly in perception and decision-making systems. The primary objective of the community is to enable vehicles to make informed decisions and safely navigate complex road conditions without human intervention. To achieve this vehicles must be capable of developing a comprehensive understanding of the environment and given the ability to make high-fidelity predictions about the future evolution of the driving scene.<a name="cite_ref-1"></a>[<sup>1</sup>](#cite_note-1) One of the major challenges lies in the accurate detection and prediction of adverse events, such as the appearance and recognition of occluded vehicles and sudden unexpected behaviors of the world environment.<a name="cite_ref-1"></a>[<sup>1</sup>](#cite_note-1)

To address these challenges, researchers have developed end-to-end algorithm frameworks that allow autonomous vehicles to dynamically adapt to complex and unpredictable environments. Many of these methods focus on the behavior of experts combined with learning features of the driving scene to make predictions, plan trajectories, and take actions. Broadly speaking, these approaches fall into two main categories: trajectory-based controllers and direct control prediction. <a name="cite_ref-2"></a>[<sup>2</sup>](#cite_note-2)

Trajectory-based methods leverage intermediate representations—such as vector space representations or bird’s-eye view maps—to decouple perception from control. By first constructing a structured understanding of the driving scene, these methods enable safer and more interpretable decision-making. This modular approach facilitates the application of traditional control techniques, such as Model Predictive Control (MPC) or Rule Based methods, and allows for easier integration of safety checks, as errors can be localized within a specific module. [<a name="cite_ref-3"></a>[<sup>3</sup>](#cite_note-3)<a name="cite_ref-7"></a>,[<sup>7</sup>](#cite_note-7)] Furthermore, trajectory-based systems can incorporate probabilistic forecasting models to anticipate the motion of other agents in the scene, reducing the risk of unexpected interactions.<a name="cite_ref-4"></a>[<sup>4</sup>](#cite_note-4)

In contrast, direct control prediction approaches bypass the need for explicit intermediate representations by mapping raw sensor inputs directly to low-level commands—such as steering, acceleration, and braking. This end-to-end paradigm enables joint optimization of the entire pipeline, which can yield a more simple and efficient system. Researchers are also exploring hybrid frameworks that combine the strengths of both methods: using intermediate representations to provide structured safety checks while also leveraging direct control predictions for faster, more adaptable decision-making in unpredictable real-world scenarios.<a name="cite_ref-2"></a>[<sup>2</sup>](#cite_note-2)

Studying autonomous vehicle capabilities is important to improve road safety, reduce traffic congestion, and enhance mobility. However, real-world testing of autonomous vehicles presents significant challenges, including high costs, safety risks in uncontrolled urban environments, and the need for policy adaptations by local governments<a name="cite_ref-5"></a>[<sup>5</sup>](#cite_note-5).

Simulation platforms, like the CARLA simulator, offer a safe, repeatable, and scalable playground where researchers can evaluate autonomous driving algorithms across a diverse array of scenarios.<a name="cite_ref-6"></a>[<sup>6</sup>](#cite_note-6) By recreating adverse weather conditions, complex traffic patterns, and rare corner cases within a controlled virtual environment, simulation enables rapid prototyping and iterative refinement of both trajectory-based and end-to-end control methods.

# Problem Statement
We aim to create an AI agent that can autonomously drive in a realistic simulation environment (CARLA) by processing sensor data and making safe control decisions. The problem is defined as an MDP with observable states (camera images), discrete actions (vehicle control commands), and rewards that penalize collisions, red-light violations, and idleness while rewarding progress toward a destination. The task is quantifiable (via cumulative reward and collision counts), measurable (through defined evaluation metrics), and replicable (using standardized simulation scenarios).

The challenge is to learn an autonomous driving policy that accurately predicts control actions at each time step(throttle, brake, and steer). Formally, given a state x — an image from a single camera — the goal is to learn a policy $\pi\theta$ that outputs control actions $a = (a_t, a_{t+1}, ..., a_{t+K})$ at each time step.

The problem is defined as follows:

- Quantifiable: The system’s performance can be expressed in terms of measurable metrics such as collision rate, travel time, and traffic regulation penalties. Improvements can be quantified against existing models on [CARLA Leaderboard Benchmark](https://paperswithcode.com/sota/autonomous-driving-on-carla-leaderboard)
- Measurable: Success will be measured by the system’s ability to complete driving missions without collisions, while optimizing efficiency/travel time and obeying traffic regulations.
- Replicable: The use of a standardized simulation environment (CARLA) ensures that experimental conditions, sensor inputs, and traffic scenarios can be consistently reproduced and tested.

# Data Collection
The training data is collected in real-time through direct interaction with the CARLA simulator. The agent generates its own data by exploring the environment and learning from experience, following the reinforcement learning (RL) paradigm.

## Sensor Inputs
The primary data sources include:
- **RGB Camera Sensor:** Mounted at the front of the vehicle, capturing images at **10 FPS** with a resolution of **384x256 pixels** and a **135-degree field of view (FOV)**. 
- **Collision Sensor:** Detects collisions with other actors (vehicles, pedestrians, static objects) and records events to compute penalties based on the collision type.
- **LiDAR Sensor:** Generates **point cloud data** representing distances and intensities. 


<div style="text-align: center;">
  <img src="assets/test_camera_image.jpg" width="400" height="300" style="display:inline-block; margin: 0 10px;"> 
  <img src="assets/debug_lidar.png" width="400" height="300" style="display:inline-block; margin: 0 10px;">
  <p style="font-size: 0.9em; font-style: italic;">Figure Left: Sample RGB camera input (384x256 pixels) used as the primary state representation for the agent.</p>
  <p style="font-size: 0.9em; font-style: italic;">Figure Right: Visualization of LiDAR point cloud data providing depth information.</p>
</div>



Data is collected in rollouts of up to 20 timesteps, where each rollout consists of state-action-reward-value tuples. The camera sensor captures images at fixed intervals (**0.1 seconds**), which are used as input for the neural network to predict action probabilities and state values.

## Image Processing
The raw camera images undergo a preprocessing pipeline:
1. Conversion from a raw buffer to a NumPy array.
2. Removal of the alpha channel, keeping only RGB values.
3. Transformation into a PyTorch tensor and transposition to set channels as the first dimension.
4. **Normalization:** Pixel values are scaled to [0,1].
5. **HSV Color Thresholding:** To detect red lights, the image is converted from BGR to HSV color space, and two red masks are applied to cover both ends of the hue spectrum. The central region is analyzed, and a red light is detected if red pixel concentration exceeds 20%.

### Dataset Characteristics
The dataset is generated as the agent interacts with the CARLA environment. A prioritized experience replay (PER) buffer enhances sample efficiency by storing transitions with higher temporal-difference (TD) errors, see the [Supplementary Materials](#Supplementary-Materials) section.. 

### State Representation:
The primary input to the neural network consists of processed camera images, providing a visual representation of the environment for decision-making.


# Proposed Solution

The solution implements a deep reinforcement learning approach using an Advantage Actor-Critic (A3C) architecture. This approach enables end-to-end learning from raw camera inputs to vehicle control commands through direct interaction with the environment.

The model consists of an Actor (Policy) Network which determines which actions to take in a given state; and a Critic (Value) Network which estimates the expected return from each state. These components share convolutional layers to process visual inputs while maintaining separate output heads for policy and value estimation. For detailed network architecture, see the [Supplementary Materials](#Supplementary-Materials) section.

The network processes 384×256 RGB images and outputs logits for 6 discrete actions (steer left, steer right, throttle, brake, maintain speed, reverse) along with a value estimate of the current state.

Our implementation uses a synchronous version of the Advantage Actor-Critic  algorithm with the following components:
1. Rollout Collection: The agent interacts with the environment for 20 timesteps, collecting state-action-reward-value tuples.
Multi-step Returns: For each state in the rollout, a bootstrapped return is calculated using:
   $$R_t = r_t + \gamma r_{t+1} + \gamma^2 r_{t+2} + ... + \gamma^n V(s_{t+n})$$
  
  where $\gamma =0.9$ .

2. Advantage Calculation: The advantage is computed as the difference between the return and the value estimate:
   $$A(s_t, a_t) = R_t - V(s_t)$$

3. Loss Function: The network is updated by minimizing:
   $$L = L_{\text{policy}} + \beta \cdot L_{\text{value}} - \alpha \cdot L_{\text{entropy}}$$

where:
$$L_{\text{policy}} = -\log(\pi(a_t|s_t)) \cdot A(s_t, a_t)$$
$$L_{\text{value}} = (V(s_t) - R_t)^2$$
$$L_{\text{entropy}} = -\sum \pi(a|s_t) \cdot \log(\pi(a|s_t))$$
$$\beta = 0.5 \text{ (value loss coefficient)}$$
$$\alpha = 0.05 \text{ (entropy coefficient)}$$

We utilize a prioritized experience replay where transitions are stored with priorities based on their TD errors, allowing more informative experiences to be sampled more frequently during off-policy updates.

### Reward Function Design
The reward function integrates multiple aspects of driving behavior:
- **Speed Compliance:** Rewards are given for maintaining a safe speed.
- **Collision Penalties:** Negative rewards are applied for collisions, with severity based on the type of collision.
- **Idling and Reversing Penalties:** Penalizes excessive idling or reversing.
- **Progress Toward Destination:** Rewards are assigned based on the decrease in distance to the target, with a large reward for reaching the goal.
- **Traffic and Lane Compliance:** Negative rewards are given for violations such as running red lights or deviating from the lane.

### Action Selection:
The Actor-Critic network outputs a stochastic policy, and actions are chosen using epsilon-greedy exploration. Possible actions include:
- Steering (Lateral)
- Throttling (Longitudinal)
- Braking (Longitudinal)
- Reversing (Longitudinal)

### Learning Through Rewards:
The agent refines its policy to optimize the return of rewards at the current timestep. Training updates use temporal difference learning to optimize the agent's behavior over time.

# Evaluation Metrics

We evaluate our reinforcement learning agent using several metrics that capture different aspects of autonomous driving performance:

## 1. Success Rate
The primary metric is the success rate, defined as the percentage of episodes where the agent successfully reaches the destination without collisions or traffic violations:

$$\text{Success Rate} = \frac{\text{Number of successful episodes}}{\text{Total number of episodes}} \times 100\%$$

A successful episode requires the agent to navigate from start to destination while maintaining safety and following traffic rules.

## 2. Collision Rate
This metric measures the frequency of collisions per episode:

$$\text{Collision Rate} = \frac{\text{Total number of collisions}}{\text{Total number of episodes}}$$

Lower values indicate safer driving behavior. We further categorize collisions by type (vehicle, pedestrian, static object) to provide more detailed analysis.

## 3. Cumulative Reward
The total reward accumulated over an episode reflects the overall quality of the driving policy:

$$\text{Cumulative Reward} = \sum_{t=0}^{T} r_t$$

where $r_t$ is the reward at timestep $t$ and $T$ is the episode length. Higher values indicate better performance across all reward components (progress, safety, rule compliance).

## 4. Average Speed
This metric evaluates the efficiency of the driving policy:

$$\text{Average Speed} = \frac{1}{T} \sum_{t=0}^{T} v_t$$

where $v_t$ is the vehicle speed at timestep $t$. This helps assess whether the agent maintains appropriate speeds without excessive stopping or speeding.

## 5. Traffic Rule Violation Rate
This measures the frequency of traffic violations per episode:

$$\text{Violation Rate} = \frac{\text{Number of traffic violations}}{\text{Total number of episodes}}$$

Violations include running red lights and lane departures, with lower values indicating better rule compliance.

These metrics collectively provide a comprehensive evaluation of the agent's performance, balancing safety, efficiency, and goal achievement in autonomous driving tasks.

# Results

Our analysis focuses on the performance of the A3C reinforcement learning agent in the CARLA simulator environment. We present results from training over XXX episodes.

## Training Performance

The primary challenge in training our agent became evident through the cumulative reward trajectory. As shown in Figure 1, the agent struggled to achieve consistent positive rewards.

<div style="text-align: center;">
    <img src="assets/reward_graph_episode_6000.png" width="500" alt="Training Rewards">
    <p><em>Figure 1: Cumulative reward at episode 6000 during training.</em></p>
</div>

The reward volatility reflects the complexity of the autonomous driving task when learning directly from pixels. As shown in Figure 1, the rewards spike frequently and struggle to stabilize, indicating the agent's difficulty in converging on a consistent driving strategy. Despite training for 6000 episodes, the agent was unable to develop an optimal policy; likely due to constraints in resources and training time. The high-dimensional nature of learning from raw camera inputs requires substantially more training iterations and more complex models. Occasional positive reward spikes occurred when the agent made progress toward destinations, but these were typically followed by significant penalties from collisions or traffic violations, preventing the accumulation of stable learning signals.

## Success and Collision Rates

As illustrated in Figure 2, we did not achieve high success rates. 

![Success Rate](assets/success_rate.png)
*Figure 2: Success rate over training episodes. The agent achieved successful navigation in less than 0.2% of episodes even after extensive training.*

Additionally, the collision rate remained high, as shown in Figure 3.

![Collision Rate](assets/collision_rate.png)
*Figure 3: Average number of collisions per episode. Despite some reduction in collisions over time, the rate remained high (80-140 collisions per episode).*

## Action Distribution Analysis

To better understand the agent's behavior, we analyzed the distribution of actions selected during later training episodes. Figure 4 shows this distribution after 5,000 episodes.

![Action Distribution](assets/action_distribution.png)
*Figure 4: Distribution of actions selected by the agent. The policy shows a bias toward throttle and steering actions, with limited use of braking.*

This distribution reveals that the agent developed a preference for forward motion (throttle) combined with steering, but rarely utilized braking effectively. This partially explains the high collision rate, as the agent failed to learn appropriate stopping behavior in response to obstacles.

## Visual Processing Capabilities

We examined the agent's visual processing by visualizing activation maps from the convolutional network layers using Gradient-weighted Class Activation Mapping (GradCAM). Figure 5 shows sample camera input alongside the corresponding attention visualization.


![Visual Processing](assets/visual_attention.png)
*Figure 5: Camera input (left) and corresponding network attention visualization (right). The attention maps suggest the network focuses on road boundaries but struggles with obstacle detection.*

<INSERT RESULTS HERE>

## Learning Curve Analysis

Finally, we analyzed the agent's learning curve by plotting the average value estimates against actual returns, as shown in Figure 6.

![Value Estimation](assets/value_estimation.png)
*Figure 6: Comparison of predicted state values versus actual returns. The persistent gap indicates the agent's difficulty in accurately estimating state values.*

<INSERT RESULTS HERE>

In summary, our results demonstrate that while the A3C agent showed some learning capability, it failed to master the complex task of autonomous driving from raw pixels. The high-dimensional state space, delayed rewards, and complex dynamics of the driving environment posed significant challenges for end-to-end reinforcement learning.

# Discussion

### Limitations

The current implementation shows limited success in achieving its goal, with training metrics indicating a very low success rate (0.0-0.2%) even after 6000 episodes. The agent consistently experiences a high number of collisions. There are several limitations. Relying solely on a front-facing RGB camera creates perceptual limitations including blind spots and lack of depth perception. Learning directly from raw pixel inputs presents challenges due to the extremely high-dimensional state space and difficulty in extracting relevant features without extensive training. The reward structure provides insufficient learning signals, with the fixed rollout length of 20 steps being too short for meaningful learning in complex driving scenarios.. Additionally, real-time decision-making requirements constrain model complexity, and the lack of prior knowledge or encoded driving rules forces the agent to learn safety-critical behaviors purely from trial and error, which is highly inefficient.

### Future work
Future work should focus on several key improvements to address the current limitations:

1. **Curriculum Learning**: Implement a structured progression of driving scenarios with increasing complexity:
   - Start with basic straight-road navigation without obstacles
   - Gradually introduce simple turns and intersections without traffic
   - Add stationary obstacles before moving vehicles
   - Finally introduce complex urban environments with traffic and pedestrians

2. **Imitation Learning Integration**:
   - Collect expert demonstrations in the CARLA environment
   - Pre-train the network using behavioral cloning to learn basic driving skills
   - This hybrid approach could significantly accelerate learning and provide a better initialization point before pure RL exploration

3. **Architecture Enhancements**:
   - Implement recurrent layers (LSTM/GRU) to capture temporal dependencies in driving sequences
   - Explore attention mechanisms to help the model focus on relevant parts of the visual input
   - Incorporate auxiliary prediction tasks (depth estimation, semantic segmentation) as additional learning signals
   - Experiment with larger network architectures with more capacity for feature extraction

4. **Safety Constraints**:
   - Implement constrained reinforcement learning to enforce safety requirements
   - Integrate rule-based safety monitors that can override unsafe actions
   - Explore safe exploration techniques that limit catastrophic failures during training

### Ethics & Privacy
While this project uses simulated data, deploying similar autonomous driving systems in the real world raises significant ethical concerns regarding safety, liability, and decision-making in unavoidable accident scenarios. The reward function design implicitly encodes value judgments about the relative importance of different driving behaviors (e.g., weighing progress toward destinations against safety considerations), which requires careful ethical scrutiny. If deployed, such systems would need transparent decision-making processes and clear accountability frameworks for when accidents occur. Additionally, real-world implementations would involve collecting massive amounts of camera data from public spaces, raising privacy concerns that would require appropriate anonymization techniques and data governance policies.

### Conclusion
This project demonstrates both the potential and significant challenges of applying end-to-end reinforcement learning to autonomous driving tasks. Despite implementing state-of-the-art techniques like Actor-Critic networks and prioritized experience replay, the results show that learning directly from pixels remains extremely difficult in complex, dynamic environments like CARLA. The poor performance metrics highlight the need for hybrid approaches that combine reinforcement learning with more structured representations, imitation learning, and explicit safety constraints. Future work should focus on decomposing the driving task into more manageable sub-problems and incorporating domain knowledge to guide learning, rather than relying solely on end-to-end optimization from raw sensory inputs.

# Footnotes
[1] <a name="cite_note-1"></a> [^](#cite_ref-1) Hao Shao, Letian Wang, Ruobing Chen, Steven L. Waslander, Hongsheng Li, Yu Liu. (2023). ReasonNet: End-to-End Driving with Temporal and Global Reasoning.

[2] <a name="cite_note-2"></a> [^](#cite_ref-2) Penghao Wu, Xiaosong Jia, Li Chen, Junchi Yan, Hongyang Li, Yu Qiao. (2022). Trajectory-guided Control Prediction for End-to-end Autonomous Driving: A Simple yet Strong Baseline.

[3] <a name="cite_note-3"></a> [^](#cite_ref-3) Hongyan Guo, Dongpu C., Hong C., Zhenping S., Yungfeng H. (2019). Model predictive path following control for autonomous cars considering a measurable disturbance: Implementation, testing, and verification.


[4] <a name="cite_note-4"></a> [^](#cite_ref-4) Jinxin Lui, Yugong Luo, Zhihua Z., Kequiang Li, Heye H., Hui X. (2022). A Probabilistic Architecture of Long-Term Vehicle Trajectory Prediction for Autonomous Driving.

[5] <a name="cite_note-5"></a> [^](#cite_ref-5) Faisal, A., Kamruzzaman, M., Yigitcanlar, T., & Currie, G. (2019). Understanding autonomous vehicles: A systematic literature review on capability, impact, planning and policy. Journal of Transport and Land Use, 12(1), 45–72.

[6] <a name="cite_note-6"></a> [^](#cite_ref-6) Ros, G., Koltun, V., & Lopez, A. M. (2017). Carla simulator. Introduction - CARLA Simulator.

[7]  <a name="cite_note-7"></a> [^](#cite_ref-7) Shao, Hao, et al. (2022) Safety-Enhanced Autonomous Driving Using Interpretable Sensor Fusion Transformer.

# Supplementary Materials

## Network Architecture
The neural network follows a **Convolutional Actor-Critic** architecture with shared feature extraction layers and separate **Actor** and **Critic** heads.

### Convolutional Layers:
- **Conv1:** 3 → 16 channels, **5×5** kernel, **stride 2**
- **MaxPool:** **2×2**
- **Conv2:** 16 → 32 channels, **5×5** kernel, **stride 2**
- **MaxPool:** **2×2**

### Fully Connected Layers:
- **FC Common:** 32×15×23 → **128 neurons**
- **Actor Head:** 128 → **6 actions**
- **Critic Head:** 128 → **1 value**

## Experience Replay Buffer:
- **Buffer Capacity:** 100,000 transitions.
- **Batch Size:** 256 transitions per update.
- **Sampling Strategy:** Higher priority given to experiences with greater TD errors for improved off-policy learning.

## Reward Function Components

### **Speed Compliance**
- **+0.3** for maintaining safe speeds  
- **-0.3** for exceeding speed limits  

### **Collision Penalties**
- **-5 to -40**, based on collision severity (e.g., pedestrian vs. static object)  

### **Progress Reward**
- **+1.5 × (previous_distance - current_distance)**  
- Encourages continuous movement toward the goal  

### **Destination Bonus**
- **+2000** upon reaching the destination  

### **Traffic Light Penalty**
- **-5.0** for running a red light  

### **Lane Keeping Penalty**
- **-0.1 × (heading_difference - 15°)** for deviations from the lane  

## Exploration Strategy
The agent follows an **epsilon-greedy strategy** with gradual decay to balance exploration and exploitation:

- **Initial ε = 1.0** (100% random actions)
- **Minimum ε = 0.2** (20% random actions)
- **Decay rate = 0.9999** per episode

## Implementation Details
The implementation is built using the following Python libraries:

- **PyTorch**: Neural network training and optimization  
- **CARLA**: Autonomous driving simulation  
- **NumPy**: Numerical computations  
- **OpenCV**: Image preprocessing and traffic light detection  

This structured approach ensures the agent learns effective driving policies while adhering to safety constraints.
